found so i can do in google colab


In [ ]:
# Install Java, Spark 3.4.1 (working stable), and findspark
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

# Initialize findspark
import findspark
findspark.init()


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


import

In [27]:
from pyspark.sql import SparkSession
from time import time
import matplotlib.pyplot as plt
from pyspark.sql.functions import regexp_replace, col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import LinearRegression
import pandas as pd
import numpy as np


spark = SparkSession.builder.appName("a5").getOrCreate()


dataset

In [3]:
import kagglehub

# Download latest version
file = kagglehub.dataset_download("danielcalvoglez/us-tariffs-2025")

print("Path to dataset files:", file)

100%|██████████| 8.89k/8.89k [00:00<00:00, 10.7MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/danielcalvoglez/us-tariffs-2025/versions/2


In [13]:
df = spark.read.csv(file, header=True, inferSchema=True, sep=";")
df.show()



+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|            Country|US 2024 Deficit|US 2024 Exports|US 2024 Imports (Customs Basis)|Trump Tariffs Alleged|Trump Response|Population|
+-------------------+---------------+---------------+-------------------------------+---------------------+--------------+----------+
|        Afghanistan|          -11.1|           11.4|                           22.6|                  49%|           25%|  41454761|
|            Albania|           13.4|          141.7|                          128.3|                  10%|           10%|   2745972|
|            Algeria|      -1,447.10|       1,014.50|                       2,461.60|                  59%|           29%|  46164219|
|            Andorra|            1.5|            4.9|                            3.4|                  10%|           10%|     80856|
|             Angola|      -1,186.90|          682.4|         

In [14]:
df.columns

['Country',
 'US 2024 Deficit',
 'US 2024 Exports',
 'US 2024 Imports (Customs Basis)',
 'Trump Tariffs Alleged',
 'Trump Response',
 'Population']

In [19]:
df = df.withColumn("Trump Tariffs Alleged", regexp_replace("Trump Tariffs Alleged", "%", "").cast("float"))
df = df.withColumn("Trump Response", regexp_replace("Trump Response", "%", "").cast("float"))

In [21]:
X = df[["Trump Tariffs Alleged"]]
y = df[["Trump Response"]]

In [22]:
X

DataFrame[Trump Tariffs Alleged: float]

In [23]:
Y

DataFrame[Trump_Response: string]

In [24]:
trainDF, testDF = df.randomSplit([.8,.2],seed=42)

In [25]:
assembler = VectorAssembler(inputCols=["Trump Tariffs Alleged"], outputCol="features")
trainDF = assembler.transform(trainDF)
testDF = assembler.transform(testDF)

In [28]:
lr = LinearRegression(featuresCol="features", labelCol="Trump Response")
model = lr.fit(trainDF)


In [31]:
prediction = model.transform(testDF)
prediction.select("Trump Tariffs Alleged", "Trump Response", "prediction").show(10)

print("RMSE:", model.summary.rootMeanSquaredError)


+---------------------+--------------+------------------+
|Trump Tariffs Alleged|Trump Response|        prediction|
+---------------------+--------------+------------------+
|                 59.0|          29.0|30.104156402306927|
|                 10.0|          10.0| 9.537606552954635|
|                 10.0|          10.0| 9.537606552954635|
|                 10.0|          10.0| 9.537606552954635|
|                 10.0|          10.0| 9.537606552954635|
|                 74.0|          37.0|  36.4000390092515|
|                 88.0|          44.0| 42.27619610906644|
|                 10.0|          10.0| 9.537606552954635|
|                 22.0|          11.0|14.574312638510298|
|                 10.0|          10.0| 9.537606552954635|
+---------------------+--------------+------------------+
only showing top 10 rows

RMSE: 1.1768521597896893
